# 匯入套件

In [1]:
import requests
import urllib
import json
from jsonpath import jsonpath
import jmespath
import pandas as pd
from datetime import datetime,timedelta
import time

# HTTP 請求參數與鏈接

In [2]:
def get_link_headers():
    keyword = '桌遊'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68',
        'x-api-source': 'pc',
        'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
    }
    
    s = requests.Session()
    url = 'https://shopee.tw/api/v4/search/product_labels'
    r = s.get(url, headers=headers)
    return s,headers

# 獲取api內容

In [3]:
def get_api_content(url):
    s,headers=get_link_headers()
    r = s.get(url, headers=headers)
    clean_needed = r.text
    json_data = json.loads('['+clean_needed+']')
    return json_data

# 商店部分

In [6]:
def shop():
    time=0
    for shoptime in df['商店api']:
        temp=jmespath.search('[].data.name',get_api_content(shoptime))
        df.loc[time,'店鋪名稱']=temp[0]
        time+=1
    return df

# 商品部分

In [7]:
def item():
    time=0
    for itemtime in df['商品api']:
        temp0=jmespath.search('[].item.models[].price',get_api_content(itemtime))
        df.loc[time,'選項售價']=",".join(str(x/100000) for x in temp0)
        temp1=jmespath.search('[].item.models[].sold',get_api_content(itemtime))
        df.loc[time,'選項銷量']=",".join(str(x) for x in temp1)
        time+=1
    return df

In [4]:
final_data = []
for time in range(1):
    time*=50
    url = f'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=桌遊&limit=50&newest={time}&order=desc&page_type=search&version=2'
    result=jmespath.search('[].items[*].{"店家帳號":shopid,"上架時間":ctime,"上架天數":ctime,"itemID":itemid,"商品標題":name,"商品網址":[name,shopid,itemid],"總銷售件數":historical_sold,"30天內銷售件數":sold,"售價":[price_min,price_max],"平均日銷(總銷件/上架天數)":[historical_sold,ctime],"喜歡數":liked_count,"消費潛力(總銷件/喜歡數)":[historical_sold,liked_count],"評價數":cmt_count,"評價綜合分數":item_rating.rating_star,"給評率(幾個評價/總銷件)":[cmt_count,historical_sold],"商品選項":tier_variations[0].options,"商品api":[itemid,shopid],"商店api":shopid}',get_api_content(url))
    final_data.append(result)
df = pd.DataFrame(final_data[0][0])

In [5]:
df['上架時間']=df['上架時間'].apply(lambda x:datetime.fromtimestamp(x))
df['上架天數']=df['上架天數'].apply(lambda x:(datetime.now()-datetime.fromtimestamp(x)).days)
df['商品網址']=df['商品網址'].apply(lambda x:"http://shopee.tw/"+str(x[0])+"-i."+str(x[1])+'.'+str(x[2]))
df['售價']=df['售價'].apply(lambda x:[int(a/100000) for a in x])
df['平均日銷(總銷件/上架天數)']=df['平均日銷(總銷件/上架天數)'].apply(lambda x:(x[0]/(datetime.now()-datetime.fromtimestamp(x[1])).days))
df['消費潛力(總銷件/喜歡數)']=df['消費潛力(總銷件/喜歡數)'].apply(lambda x:0 if x[1]==0 else (x[0]/x[1]))
df['給評率(幾個評價/總銷件)']=df['給評率(幾個評價/總銷件)'].apply(lambda x:0 if x[1]==0 else (x[0]/x[1]))
df['商品api']=df['商品api'].apply(lambda x:"https://shopee.tw/api/v2/item/get?itemid="+str(x[0])+"&shopid="+str(x[1]))
df['商店api']=df['商店api'].apply(lambda x:"https://shopee.tw/api/v4/product/get_shop_info?shopid="+str(x))

In [8]:
for time in range(len(df)):
    df.loc[time,'售價']=",".join(str(x) for x in df.loc[time,'售價'])
    df.loc[time,'商品選項']=",".join(x for x in df.loc[time,'商品選項'])

In [11]:
today = time.strftime("%Y%m%d")
df.to_csv(f"{today}.csv",index=False)